First, we manage all imports for the code.

In [79]:
!pip install gensim
!pip install pandas
!pip install nltk
!pip install install -Uq bertopic

ERROR: Could not install packages due to an OSError: [WinError 5] Zugriff verweigert: 'C:\\Users\\k-mar\\AppData\\Local\\Programs\\Python\\Python310\\Lib\\site-packages\\~umpy\\.libs\\libopenblas64__v0.3.21-gcc_10_3_0.dll'
Consider using the `--user` option or check the permissions.



In [83]:
#imports
import pandas as pd
import numpy as np
import random
import ast
from torch import nn
import torch
import re
import nltk
from torch.utils.data import random_split, Subset, DataLoader
import gensim
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

import zipfile
import os
import shutil

#import bertopic
from sentence_transformers import SentenceTransformer

Next we process the zipped data to csv. It is stored in .zip, because it is smaller on git and then locally it is extracted to csv.

In [84]:
with zipfile.ZipFile("data/dataset.zip", "r") as zip_ref:
    zip_ref.extractall("data")

for file_name in os.listdir("data/dataset"):
    source = "data/dataset/" + file_name
    destination = "data/" + file_name
    if os.path.isfile(source): shutil.move(source, destination)

os.rmdir("data/dataset")
del source, destination

Preprocessing of the datasets, the goal is to receive a table with the following columns: Name, Lyrics, Genre<br>
df Dataset - Name, Lyrics, Genre<br>
df3 Dataset - 10000 entrys of random selectet lyrics to train doc2vec

In [89]:

df = pd.read_csv('data/Spotify-2000.csv')
df = df[['Title', 'Top Genre']] #take only the name and genre
df2 = pd.read_csv('data/spotify_millsongdata.csv')
df['lyrics'] = '' # add column lyrics
#Now combine those two documents by the title
found = 0

for x, title in enumerate(df['Title']):
    #print(title)
    for y, title2 in enumerate(df2['song']):
        if title2.lower() == title.lower():
            df['lyrics'].iloc[x] = df2['text'].iloc[y]

print("finished combining..")

finished combining..


Collect 10000 random entrys of lyrics from the millsongdata Dataset

In [90]:
df3 = df2['text'].sample(n=10000)
print(df3)
print('done')

43142    I've been the needle and the thread  \r\nWeavi...
40129    See I was born a little pie-eyed motherfucker ...
22719    If you been out messin' and actin' a fool  \r\...
13555    All eyes on her center of all attention  \r\nA...
28905    You could sail across the water take the kids ...
                               ...                        
4923     I am starting to function  \r\nIn the usual wa...
900      If fear's what makes us decide  \r\nOur future...
36319    My life - I'm a fool for you  \r\nYou who take...
15903    It's too late  \r\nThis model's out of date  \...
44042    Chapter one we started happy  \r\nThe second t...
Name: text, Length: 10000, dtype: object
done


In [91]:
#remove songs which were not in both datsets
songs_to_remove = []
for x, lyrics in enumerate(df['lyrics']):
    if lyrics == '':
        songs_to_remove.append(x)
df.drop(songs_to_remove, axis = 0, inplace = True)

print(df)

                  Title          Top Genre  \
0               Sunrise    adult standards   
1           Black Night         album rock   
3         The Pretender  alternative metal   
6     She Will Be Loved                pop   
8        Mr. Brightside        modern rock   
...                 ...                ...   
1988         Summertime    adult standards   
1989   Heartbreak Hotel    adult standards   
1990          Hound Dog    adult standards   
1991    Johnny B. Goode         blues rock   
1993     Blueberry Hill    adult standards   

                                                 lyrics  
0     You take away the breath I was keeping for sun...  
1     Black night is not right,  \r\nI don't feel so...  
3     I'm going to rent myself a house  \r\nIn the s...  
6     Beauty queen of only eighteen she  \r\nHad som...  
8     Coming out of my cage  \r\nAnd I've been doing...  
...                                                 ...  
1988  Summertime....and the livin' is eas

Further preprocessing of the lyrics itself. We remove the stopwords and punctuations with regex and stopwords from ntlk form df and df3

In [92]:
from nltk.corpus import stopwords
nltk.download('stopwords')

sw = stopwords.words('english')
punc_regex = r'[^\s\w]' #searches for everything thats not a word or space
stopword_regex = r'\b{0}\b'
space_regex = r'\s\s+'
newl_regex = r'\n|\r'

print(df3.iloc[1])

#preprocessing of df
for x, lyrics in enumerate(df['lyrics']):
    txt = lyrics
    txt = re.sub(punc_regex, '', txt) #remove punuctuations
    for sword in sw:
        txt = re.sub(stopword_regex.format(sword), '', txt, flags=re.IGNORECASE) #remove every stopword
    txt = re.sub(space_regex, ' ', txt)
    df['lyrics'].iloc[x] = txt


    
#preprocessing of df3
for x, lyrics in enumerate(df3):
    txt = lyrics
    txt = re.sub(punc_regex, '', txt) #remove punuctuations
    for sword in sw:
        txt = re.sub(stopword_regex.format(sword), '', txt, flags=re.IGNORECASE) #remove every stopword
    txt = re.sub(space_regex, ' ', txt)
    df3.iloc[x] = txt
    
#print(df)
print(df3.iloc[1])

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\k-mar\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


See I was born a little pie-eyed motherfucker  
Mama she left me and my papa was a hard trucker  
Out on the highway we loved to roll  
He never made me go to school  
  
I never begged to go  
I was a low class livin' raised out in the sticks  
I was born to be a hick  
See I love to spend my days just a squirrel huntin'  
  
Go see my cousin Ellie May and get some good lovin'  
Kissin' and huggin' on some distant lands  
People always tell me I'm a twisted man  
Jim Beam in my hand boones kegged in shit  
  
And I was born to be a hick  
Ohh I was born to be a hick  
  
See I love to spend my days just a squirrel hunter  
Go see my cousin Ellie May and get some good lovin  
Kissin' and huggin' on some distant lands  
People always tell me I'm a twisted man  
Jim Beam in my hand boones kegged in shit  
  
And I was born to be a hick  
See I was born to be a hick man  
Yeah I was born to be a hick man  
Yeah, yeah, yeah  
  
I'm a shotgun tokin'  
I'm a John Deere drivin'  
I'm a hick 

In [ ]:
sentence_model = SentenceTransformer("all-MiniLM-L6-v2")

#display(df)

topic_model = bertopic.BERTopic(language='english', embedding_model=sentence_model,verbose=True)

topics, probs = topic_model.fit_transform(df["lyrics"])

print(topics)

In [ ]:
print(topic_model.get_topic_info())

print(topic_model.get_topics()[1])

Tokenize the lyrics and create tagged Documents

In [116]:
tagged_data = []
nltk.download('punkt')

for i,d in enumerate(df3):
    tokenized_words = nltk.tokenize.word_tokenize(d)
    tagged_data.append(TaggedDocument(words=tokenized_words, tags=str(i)))

print(len(tagged_data))

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\k-mar\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


10000


Setting up the Doc2Vec model

In [117]:
doc2vec_model = gensim.models.doc2vec.Doc2Vec(vector_size=100, min_count=1, epochs=30)
doc2vec_model.build_vocab(tagged_data)
doc2vec_model.train(tagged_data, total_examples=doc2vec_model.corpus_count, epochs=doc2vec_model.epochs)

print("Doc2Vec model finished training")

Doc2Vec model finished training


Building the DataLoader for the machine learning model

In [143]:
columns = ['Token'] + ['Target']
df_for_dataloader = pd.DataFrame(columns = columns)
df_for_dataloader.set_index(columns)

lookup_dict = {}
token_list = []
target_list = []
i = 0
for d in df["Top Genre"]:
    if d not in lookup_dict:
        lookup_dict[d] = i
        i+=1

for index, df_row in df.iterrows():
    lyrics_tokenized = nltk.tokenize.word_tokenize(df_row["lyrics"])
    token = [doc2vec_model.infer_vector(lyrics_tokenized)]
    #print(token)
    one_hot_encoded_vector = []
    for x in lookup_dict.keys():
        if df_row["Top Genre"] == x:
            one_hot_encoded_vector.append(1)
        else:
            one_hot_encoded_vector.append(0)
    target = [one_hot_encoded_vector] # should be genre
    #row = pd.DataFrame([token + target], columns=['Token', 'Target'])
    #df_for_dataloader = pd.concat([df_for_dataloader, row])
    token_list.append(token)
    target_list.append(target)

    
dataloader = torch.utils.data.DataLoader(list(zip(token_list,target_list)), batch_size=16)

In [144]:
device = torch.device('cpu') #torch.device('cuda' if torch.cuda.is_available() else 'cpu') For faster training in the end

class SongModel(nn.Module):
    def __init__(self, input_size, output_size):
        super(SongModel, self).__init__()
        self.input_size = input_size
        self.hidden_size = input_size * 3
        self.output_size = output_size
        self.layer_1 =   nn.Linear(self.input_size, self.hidden_size)
        self.layer_2 =   nn.Linear(self.hidden_size, self.hidden_size)
        self.layer_out = nn.Linear(self.hidden_size, self.output_size)
        self.softmax =   nn.Softmax()
        self.sigmoid =   nn.Sigmoid()
        self.tanh =      nn.Tanh()

        nn.init.xavier_uniform_(self.layer_1.weight)
        nn.init.zeros_(self.layer_1.bias)
        nn.init.xavier_uniform_(self.layer_2.weight)
        nn.init.zeros_(self.layer_2.bias)
        nn.init.xavier_uniform_(self.layer_out.weight)
        nn.init.zeros_(self.layer_out.bias)


    def forward(self, d):
        x = self.sigmoid(self.layer_1(d))
        x = self.tanh(self.layer_2(x))
        x = self.softmax(self.layer_out(x))
        return x

model = SongModel(1, 3).to(device)
optimizer = torch.optim.SGD(model.parameters(), lr=0.025)
loss_fn = torch.nn.CrossEntropyLoss()

In [145]:
def train(dataloader, model, loss_fn, optimizer):
    total_loss = 0
    iterations = 0
    for sample in dataloader:
        model_input = sample[0]
        should = sample[1]
        predict = model(model_input)
        loss = loss_fn(predict, should.float())
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        total_loss += loss
        iterations += 1
        if iterations % 1000 == 0:
            print(f"Predict: {predict}, Loss: {loss}")

    print(total_loss / len(dataloader))
    return

In [146]:
epochs = 5
for sample in dataloader:
    print(sample)
    break
    
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(dataloader, model, loss_fn, optimizer)
    #test(test_dataloader, model, loss_fn)

[[tensor([[-1.7212,  0.0557,  1.8315,  ..., -0.4117,  0.0923,  0.1124],
        [-0.0943, -0.9248, -0.1292,  ..., -0.4574, -0.4806, -0.0884],
        [ 0.6362, -0.0830,  1.0853,  ...,  0.6541,  0.0361,  1.5841],
        ...,
        [-0.2899, -2.7796,  1.0449,  ..., -1.3422, -0.9225, -0.7993],
        [-0.1310, -1.2314,  0.2063,  ..., -0.0978, -1.6872,  1.3913],
        [-0.0786, -2.0211,  2.2326,  ..., -0.2731, -1.8320,  0.1552]])], [[tensor([1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]), tensor([0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]), tensor([0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]), tensor([0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]), tensor([0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0]), tensor([0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0]), tensor([0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0]), tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0]), tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0]), tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

TypeError: linear(): argument 'input' (position 1) must be Tensor, not list